In [1]:
from libs.alibaba.alibaba import Alibaba
from libs.alibaba.p4p import P4P
from libs.json import JSON

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import WebDriverException
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from datetime import datetime
from bs4 import BeautifulSoup
from pyquery import PyQuery as pq

import redis
import pendulum
import arrow
import imp
import time
import re

import types
import traceback 
import threading

In [3]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('--disable-software-rasterizer')
chrome_options.add_argument('--disable-extensions')
chrome_options.add_argument('--disable-logging')
chrome_options.add_argument('--disable-infobars')
chrome_options.add_argument('--ignore-certificate-errors')
browser = webdriver.Chrome(chrome_options=chrome_options)

In [4]:
broker_url = 'redis://localhost:6379/0'
market = JSON.deserialize('.', 'storage', 'markets.json')['Eyelashes']
p4p = P4P(market, market['lid'], market['lpwd'],broker_url=broker_url, headless=False, browser=browser)

. storage markets.json
.\storage\markets.json was deserialized!
E:\glitter\Eyelashes_config [] p4p_keywords_list_recording.json
E:\glitter\Eyelashes_config\p4p_keywords_list_recording.json was deserialized!
E:\glitter\Eyelashes_config [] p4p_keywords_list_monitor.json
E:\glitter\Eyelashes_config\p4p_keywords_list_monitor.json was deserialized!


In [12]:
def crawl(self, group="all"):

    try:
        keywords = []
        # self.load_keywords('recording')
        with self.lock:
            self.load_url()
            # all_kws_count = int(self.browser.find_element_by_css_selector('a.all-kwcount span').text)
            all_kws_count = self.switch_to_group(group)

            kws_count = 0
            while True:
                css_selector = "div.keyword-manage .bp-table-main-wraper>table"
                table = self.browser.find_element_by_css_selector(css_selector)

                trs = table.find_elements_by_css_selector('tbody tr')

                has_checked = False
                for idx, tr in enumerate(trs):
                    kws_count += 1

                    print('index:', idx, len(trs), end=' > ')
                    ActionChains(self.browser).move_to_element(tr).perform()
                    id = tr.find_element_by_css_selector('td:first-child input').get_attribute('value').strip()
                    # if id not in self.keywords_list['recording']:
                    #     print('skipped_not_in_recording')
                    #     continue

                    grp = tr.find_element_by_css_selector('td:nth-child(4)').text.strip()
                    if group != 'all':
                        if group not in grp:
                            print('skipped_not_in_group', group, grp)
                            continue

                    dt = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                    kws = tr.find_element_by_css_selector('td:nth-child(3)').text.strip()

                    print(kws, end=' > ')

                    if not self.open_price_dialog(tr):
                        print('skipped_2')
                        continue

                    [prices, sponsors] = self.find_prices_and_sponsors()

                    if prices:
                        keywords.append([dt, id, kws, grp, prices, sponsors])

                        if (id not in self.recent_prices):
                            self.recent_prices[id] = []
                        self.recent_prices[id].append(prices)
                        if len(self.recent_prices[id]) > 5:
                            self.recent_prices[id].pop(0)

                    print(' >>>>>> successful end ')

                if not self.next_page():
                    break
        if keywords:
            print('==================================')
            self.save_crawling_result(keywords)

    except Exception as e:
        print('Error: ', e)
        traceback.print_exc()

p4p.crawl = types.MethodType(crawl, p4p)

In [10]:
def save_crawling_result(self, keywords):
    root = self.market['directory'] + '_config'
    date_str = keywords[0][0].split(' ')[0]
    fn = 'p4p_keywords_crawl_result_'+date_str+'.json.gz'
    print(fn, root)
    JSON.serialize(keywords, root, [], fn, append=True)
    
p4p.save_crawling_result = types.MethodType(save_crawling_result, p4p)

In [20]:
def find_prices_and_sponsors(self, close=True):
    prices = []
    sponsors = None

    kws = self.browser.find_element_by_css_selector(
        '.sc-manage-edit-price-dialog span[data-role="span-keyword"]').text.strip()
    sponsors = self.find_sponsors(kws)

    while True:
        try:
            price_table_tbody_selector = ".sc-manage-edit-price-dialog table tbody,.sc-manage-edit-price-dialog p.util-clearfix"
            price_table_tbody = WebDriverWait(self.browser, 15).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, price_table_tbody_selector)))

            if price_table_tbody.tag_name == 'p':
                prices = None
                break

            for btn in price_table_tbody.find_elements_by_css_selector('a'):
                price = btn.text.strip()
                float(price)
                prices.append(price)

            self.check_balance()
            break
        except StaleElementReferenceException:
            #             self.browser.implicitly_wait(0.5)
            time.sleep(0.5)
            continue
        except ValueError:
            prices = []
            break
    if close:
        webdriver.ActionChains(self.browser).send_keys(Keys.ESCAPE).perform()
    return [prices, sponsors]
p4p.find_prices_and_sponsors = types.MethodType(find_prices_and_sponsors, p4p)

In [10]:
def load_url(self):
    while True:
        try:
            if self.alibaba is None:
                self.logger.info('open browser and login')
                alibaba = Alibaba(self.lid, self.lpwd, headless=self.headless, browser=self.browser)
                alibaba.login()
                if self.browser is None:
                    self.browser = alibaba.browser 
                self.alibaba = alibaba

            self.browser.get(self.api)
            if 'login.alibaba.com' in self.browser.current_url:
                self.logger.info('Was out, login again!')
                self.alibaba.login()
                self.browser.get(self.api)

            WebDriverWait(self.browser, 15).until(
                EC.invisibility_of_element_located((By.CSS_SELECTOR, 'div.bp-loading-panel')))

            # try to close all follow-me-popups
            while True:
                btn_close = self.browser.find_elements_by_css_selector('div.follow-me-close')
                if btn_close:
                    webdriver.ActionChains(self.browser).send_keys(Keys.ESCAPE).perform()
                    # wait 1 second to see if new popup commes
                    self.browser.implicitly_wait(1)
                    continue
                else:
                    break
            break
        except WebDriverException as e:
            if 'chrome not reachable' in str(e):
                self.logger.info('Browser Window was closed! Try to open a new browser window.')
                self.browser = None
            continue
p4p.load_url = types.MethodType(load_url, p4p)

In [ ]:
p4p.browser.set_window_size(1920, 1200)

In [ ]:
p4p.browser.save_screenshot('screenie.png')

In [4]:
group = '直通车App'

In [5]:
p4p.load_url()
p4p.switch_to_group(group)

primary 打开网址：https://i.alibaba.com
primary 等待登陆页面加载 ... ...
primary 输入登录信息， 并登录
success 成功 登录 阿里巴巴 国际站！用户名：Jeremy


12

In [6]:
p4p.crawl(group=group)

index: 0 16 > eyelash extension silk >  >>>>>> successful end 
index: 1 16 > eyelash extension wholesale >  >>>>>> successful end 
index: 2 16 > flare eyelash >  >>>>>> successful end 
index: 3 16 > flare eyelash extension >  >>>>>> successful end 
index: 4 16 > magnetic eyelashes >  >>>>>> successful end 
index: 5 16 > mink eyelashes private label >  >>>>>> successful end 
index: 6 16 > mink lashes and custom package >  >>>>>> successful end 
index: 7 16 > own brand eyelashes >  >>>>>> successful end 
index: 8 16 > private label 3d mink eyelash >  >>>>>> successful end 
index: 9 16 > real mink eyelashes >  >>>>>> successful end 
index: 10 16 > silk eyelashes >  >>>>>> successful end 
index: 11 16 > strip eyelash >  >>>>>> successful end 
index: 12 16 > synthetic eyelashes >  >>>>>> successful end 
index: 13 16 > volume eyelash >  >>>>>> successful end 
index: 14 16 > volume eyelash extension >  >>>>>> successful end 
index: 15 16 > wholesale mink eyelash >  >>>>>> successful end 
next

In [11]:
p4p.monitor(group=group)

Task Monitor start to run with group="直通车App"
switch to group: 直通车App
index: 0 12 > eyelash extension silk > make selected >  >>>>>> successful end 
index: 1 12 > eyelash extension wholesale > make selected >  >>>>>> successful end 
index: 2 12 > flare eyelash > make selected >  >>>>>> successful end 
index: 3 12 > flare eyelash extension > make selected >  >>>>>> successful end 
index: 4 12 > mink eyelashes private label > make selected >  >>>>>> successful end 
index: 5 12 > mink lashes and custom package > make selected >  >>>>>> successful end 
index: 6 12 > own brand eyelashes > make selected >  >>>>>> successful end 
index: 7 12 > private label 3d mink eyelash > make selected >  >>>>>> successful end 
index: 8 12 > strip eyelash > make selected >  >>>>>> successful end 
index: 9 12 > synthetic eyelashes > make selected >  >>>>>> successful end 
index: 10 12 > volume eyelash > make selected >  >>>>>> successful end 
index: 11 12 > volume eyelash extension > make selected >  >>>>>>

In [10]:
p4p.turn_all_off(group=group)

next_page: False


In [29]:
float('300.00') == 300

True

In [12]:
p4p.alibaba = None

In [ ]:
p4p.kws_tracking

In [ ]:
# p4p.load_url()

# css_selector = "div.keyword-manage .bp-table-main-wraper>table"
# table = p4p.browser.find_element_by_css_selector(css_selector)
# trs = table.find_elements_by_css_selector('tbody tr')
# idx = 1
# for tr in trs:
#     print()
#     print(idx, end=': ')
#     idx += 1
    
#     kws = tr.find_element_by_css_selector('td:nth-child(3)').text.strip()
#     print(kws, end=": ")
        
#     if not p4p.open_price_dialog(tr):
#         print('skipped_2')
#         idx += 1
#         continue
        
#     [prices, sponsors] = p4p.find_prices_and_sponsors()
#     print(prices, sponsors)
#     if idx > 10:
#         break

In [ ]:
tr = trs[8]
kws = tr.find_element_by_css_selector('td:nth-child(3)').text.strip()
print(kws, end=": ")

if not p4p.open_price_dialog(tr):
    print('skipped_2')
else:
#     time.sleep(1)
    prices = p4p.find_prices()

In [ ]:
market = JSON.deserialize('.', 'storage', 'markets.json')['Tools']

In [34]:
def set_sub_budget(self, sub_budget):
    self.redis.set(self.market['name']+'_p4p_sub_budget', sub_budget)

def unset_sub_budget(self):
    self.redis.delete(self.market['name']+'_p4p_sub_budget')

In [ ]:
diff = '19.63'
diff = 0-float(diff)
sub_budget = self.redis.get(self.market['name']+'_p4p_sub_budget')
if sub_budget is not None and float(sub_budget) > 0:
    if self.redis.incrbyfloat(self.market['name']+'_p4p_sub_budget', diff) < 0:
        self.redis.set(self.market['name']+'_p4p_sub_budget_overflow', True)

In [6]:
r = redis.StrictRedis.from_url(p4p.broker_url)

In [25]:
sub_budget = '90.00'
r.set(p4p.market['name']+'_p4p_sub_budget', sub_budget)

True

In [30]:
float(r.get(p4p.market['name']+'_p4p_sub_budget'))

31.11000000000001

In [ ]:
r.get(p4p.market['name']+'_p4p_sub_budget')

In [28]:
r.incrbyfloat(p4p.market['name']+'_p4p_sub_budget', -19.63)

31.11000000000001

In [21]:
r.delete(p4p.market['name']+'_p4p_sub_budget')

1

In [31]:
diff = '19.63'
diff = 0-float(diff)
diff

-19.63

In [33]:
r.delete(p4p.market['name']+'_p4p_sub_budget_overflow')

0